In [1]:
# Redes Neuronales
import numpy as np
from sklearn.datasets import load_iris, load_breast_cancer, load_wine
from sklearn.model_selection import train_test_split, KFold, LeaveOneOut
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.base import BaseEstimator, ClassifierMixin
from scipy.spatial.distance import cdist
import warnings
warnings.filterwarnings('ignore')

class RBFNeuralNetwork(BaseEstimator, ClassifierMixin):
    def __init__(self, n_centers=10, sigma=1.0):
        self.n_centers = n_centers
        self.sigma = sigma

    def fit(self, X, y):
        # Seleccionar centros aleatoriamente
        idx = np.random.choice(X.shape[0], self.n_centers, replace=False)
        self.centers = X[idx]

        # Calcular matriz de características RBF
        rbf_features = self._compute_rbf(X)

        # Entrenar perceptrón para la capa de salida
        self.mlp = MLPClassifier(hidden_layer_sizes=(), max_iter=1000)
        self.mlp.fit(rbf_features, y)
        return self

    def predict(self, X):
        rbf_features = self._compute_rbf(X)
        return self.mlp.predict(rbf_features)

    def _compute_rbf(self, X):
        distances = cdist(X, self.centers)
        return np.exp(-distances**2 / (2 * self.sigma**2))

def load_dataset(dataset_name):
    if dataset_name == "iris":
        data = load_iris()
    elif dataset_name == "breast_cancer":
        data = load_breast_cancer()
    elif dataset_name == "wine":
        data = load_wine()
    return data.data, data.target

def evaluate_holdout(X, y, classifier):
    # Dividir datos 70/30
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # Escalar datos
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Entrenar y evaluar
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)

    return accuracy_score(y_test, y_pred), confusion_matrix(y_test, y_pred)

def evaluate_kfold(X, y, classifier, k=10):
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    accuracies = []
    conf_matrices = []

    for train_idx, test_idx in kf.split(X):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        # Escalar datos
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        # Entrenar y evaluar
        classifier.fit(X_train, y_train)
        y_pred = classifier.predict(X_test)

        accuracies.append(accuracy_score(y_test, y_pred))
        conf_matrices.append(confusion_matrix(y_test, y_pred))

    return np.mean(accuracies), np.mean(conf_matrices, axis=0)

def evaluate_leave_one_out(X, y, classifier):
    loo = LeaveOneOut()
    accuracies = []
    predictions = []
    true_values = []

    for train_idx, test_idx in loo.split(X):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        # Escalar datos
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        # Entrenar y evaluar
        classifier.fit(X_train, y_train)
        y_pred = classifier.predict(X_test)

        predictions.extend(y_pred)
        true_values.extend(y_test)

    return accuracy_score(true_values, predictions), confusion_matrix(true_values, predictions)

def main():
    # Configuración de clasificadores
    mlp = MLPClassifier(hidden_layer_sizes=(10,), max_iter=1000)
    rbf = RBFNeuralNetwork(n_centers=10, sigma=1.0)

    datasets = {
        "iris": load_iris(),
        "breast_cancer": load_breast_cancer(),
        "wine": load_wine()
    }

    classifiers = {
        "MLP": mlp,
        "RBF": rbf
    }

    # Evaluar cada combinación de dataset y clasificador
    for dataset_name, dataset in datasets.items():
        print(f"\nDataset: {dataset_name}")
        X, y = dataset.data, dataset.target

        for clf_name, clf in classifiers.items():
            print(f"\nClasificador: {clf_name}")

            # Hold Out
            acc_ho, cm_ho = evaluate_holdout(X, y, clf)
            print("\nHold Out (70/30):")
            print(f"Accuracy: {acc_ho:.4f}")
            print("Matriz de confusión:")
            print(cm_ho)

            # K-Fold
            acc_kf, cm_kf = evaluate_kfold(X, y, clf)
            print("\n10-Fold Cross-Validation:")
            print(f"Accuracy promedio: {acc_kf:.4f}")
            print("Matriz de confusión promedio:")
            print(cm_kf)

            # Leave-One-Out
            acc_loo, cm_loo = evaluate_leave_one_out(X, y, clf)
            print("\nLeave-One-Out:")
            print(f"Accuracy: {acc_loo:.4f}")
            print("Matriz de confusión:")
            print(cm_loo)

if __name__ == "__main__":
    main()


Dataset: iris

Clasificador: MLP

Hold Out (70/30):
Accuracy: 0.9778
Matriz de confusión:
[[19  0  0]
 [ 0 12  1]
 [ 0  0 13]]

10-Fold Cross-Validation:
Accuracy promedio: 0.9600
Matriz de confusión promedio:
[[4.9 0.1 0. ]
 [0.  4.7 0.3]
 [0.  0.2 4.8]]

Leave-One-Out:
Accuracy: 0.9600
Matriz de confusión:
[[50  0  0]
 [ 0 46  4]
 [ 0  2 48]]

Clasificador: RBF

Hold Out (70/30):
Accuracy: 0.8000
Matriz de confusión:
[[19  0  0]
 [ 0  7  6]
 [ 1  2 10]]

10-Fold Cross-Validation:
Accuracy promedio: 0.8067
Matriz de confusión promedio:
[[4.9 0.  0.1]
 [0.  3.6 1.4]
 [0.1 1.3 3.6]]

Leave-One-Out:
Accuracy: 0.8800
Matriz de confusión:
[[50  0  0]
 [ 0 44  6]
 [ 1 11 38]]

Dataset: breast_cancer

Clasificador: MLP

Hold Out (70/30):
Accuracy: 0.9708
Matriz de confusión:
[[ 61   2]
 [  3 105]]

10-Fold Cross-Validation:
Accuracy promedio: 0.9754
Matriz de confusión promedio:
[[20.4  0.8]
 [ 0.6 35.1]]

Leave-One-Out:
Accuracy: 0.9789
Matriz de confusión:
[[205   7]
 [  5 352]]

Clasific